In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

## 2. 2. Titanic - sns.load_dataset('titanic')




### a. 타이타닉호의 승객에 대해 나이와 성별에 의한 카테고리 열인 category1 열을 만드시오. category1 카테고리는 다음과 같이 정의됨
- 20 살이 넘으면 성별을 그대로 사용한다
- 20 살 미만이면 성별에 관계없이 child” 라고 한다

In [2]:
titanic = sns.load_dataset('titanic')
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


In [3]:
def make_category3(age, sex):
    if age >= 20:
        return sex
    elif age < 20:
        return 'child'

In [4]:
titanic['category3'] = titanic.apply(lambda x : make_category3(x.age, x.sex), axis=1)
titanic[['age', 'sex', 'category3']].iloc[100:120]

,age,sex,category3
100,28.0,female,female
101,NaN,male,None
102,21.0,male,male
103,33.0,male,male
104,37.0,male,male
105,28.0,male,male
106,21.0,female,female
107,NaN,male,None
108,38.0,male,male
109,NaN,female,None


### b.타이타닉호의 승객 중 나이를 명시하지 않은 고객은 나이를 명시한 고객의 평균 나이 값이 되도록 titanic 데이터프레임을 고치시오

In [5]:
titanic['age'] = titanic['age'].fillna(titanic['age'].mean())

In [6]:
titanic[['age', 'sex', 'category3']].iloc[[101,107,109]]

,age,sex,category3
101,29.699118,male,None
107,29.699118,male,None
109,29.699118,female,None


### 성별 , 선실별 , 출발지 embark_town 별 생존율을 구하시오

In [7]:
# 성별 생존율
survived_ratio_by_sex = titanic.groupby('sex')[['survived']].mean()
survived_ratio_by_class = titanic.groupby('pclass')[['survived']].mean()
survived_ratio_by_embark = titanic.groupby('embarked')[['survived']].mean()

pd.concat((survived_ratio_by_sex, survived_ratio_by_class, survived_ratio_by_embark), axis=0).T.round(2)

,female,male,1,2,3,C,Q,S
survived,0.74,0.19,0.63,0.47,0.24,0.55,0.39,0.34


In [8]:
# d. 타이타닉호 승객을 ‘ 미성년자’, ’청년’, ’중년’, ’장년’, 노년’ 나이 그룹으로 나누고, 각 그룹별 생존율을 구하시오
# bins = [1, 20, 30, 50, 70, 100]
# labels = ["미성년자 ", 청년 ", 중년 ", 장년 ", 노년]

titanic['age_group'] = pd.cut(x=titanic.age, bins=[1, 20, 30, 50, 70, 100], labels=['미성년자', '청년' , '중년', '장년' , '노년'])
survived_ratio_by_age_group = titanic.groupby('age_group')['survived'].mean()
survived_ratio_by_age_group

age_group
미성년자    0.424242
청년      0.334152
중년      0.423237
장년      0.355932
노년      0.200000
Name: survived, dtype: float64

In [9]:
# qcut 명령으로 세 개의 나이 그룹을 만들고 , 나이 그룹별 남녀 성비와 생존율을 구하시오
titanic['age_qcut'] = pd.qcut(x=titanic.age, q=3 , labels=['young','middle','old'])
survived_ratio_by_age_qcut = titanic.groupby(['age_qcut'])['survived'].mean()
sex_ratio_by_age_qcut = titanic.groupby(['age_qcut', 'sex'])['sex'].count() / titanic.groupby(['age_qcut'])['sex'].count()

In [10]:
survived_ratio_by_age_qcut

age_qcut
young     0.411960
middle    0.335526
old       0.405594
Name: survived, dtype: float64

In [11]:
sex_ratio_by_age_qcut

age_qcut  sex   
young     female    0.405316
          male      0.594684
middle    female    0.319079
          male      0.680921
old       female    0.332168
          male      0.667832
Name: sex, dtype: float64

In [12]:
# a.배기량 (displacement) 대비 마력 (horsepower) 열 hp_per_cc 을 추가하시오
mpg = sns.load_dataset('mpg')
mpg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    392 non-null    float64
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model_year    398 non-null    int64  
 7   origin        398 non-null    object 
 8   name          398 non-null    object 
dtypes: float64(4), int64(3), object(2)
memory usage: 28.1+ KB


In [13]:
mpg['hp_per_cc'] = mpg.horsepower / mpg.displacement

In [14]:
# b.name 으로부터 manufacturer와 모델을 추출하여 새로운 열 manufacturer 와 model 을 추가하고 , name 열은 삭제하시오
mpg['manufacturer'] = mpg['name'].apply(lambda x : x.split()[0])
mpg['model'] = mpg['name'].apply(lambda x : ' '.join(x.split()[1:]))
del mpg['name']
mpg[['manufacturer', 'model']]

,manufacturer,model
0,chevrolet,chevelle malibu
1,buick,skylark 320
2,plymouth,satellite
3,amc,rebel sst
4,ford,torino
...,...,...
393,ford,mustang gl
394,vw,pickup
395,dodge,rampage
396,ford,ranger


In [19]:
mpg.groupby('manufacturer').groups['chevrolet']

Int64Index([  0,   6,  12,  30,  35,  38,  46,  60,  62,  73,  87,  91,  98,
            103, 109, 115, 128, 132, 133, 153, 157, 165, 187, 192, 195, 196,
            221, 225, 230, 237, 253, 262, 266, 285, 291, 306, 311, 313, 341,
            367, 368, 369, 392],
           dtype='int64')

In [18]:
# c.엔진의 실린더 갯수별 연비의 평균을 구하시오
mpg.groupby('cylinders')[['mpg']].mean().round(3)

,mpg
cylinders,
3,20.550
4,29.287
5,27.367
6,19.986
8,14.963


In [20]:
mpg[['mpg','cylinders']].groupby('cylinders').mean().round(3)

,mpg
cylinders,
3,20.550
4,29.287
5,27.367
6,19.986
8,14.963


In [23]:
mpg.pivot_table(index='cylinders', values='mpg').round(3)

,mpg
cylinders,
3,20.550
4,29.287
5,27.367
6,19.986
8,14.963


In [ ]:
# d.생산지별 배기량 대비 마력(hp_per_cc)의 평균을 구하시오
mpg.groupby('origin')['hp_per_cc'].mean()

origin
europe    0.740005
japan     0.789485
usa       0.512063
Name: hp_per_cc, dtype: float64

In [ ]:
# e.모델이 5 개 이상인 제조사에 대하여 연비의 평균이 가장 좋은 제조사 Top 5 를 구하시오
many_model_manufacturers = np.unique(mpg.manufacturer)[mpg.groupby('manufacturer')['model'].count()>5]

mpg.groupby('manufacturer')['mpg'].mean().sort_values()[::-1]

all_mpg = []
for i in mpg.groupby('manufacturer')['mpg'].mean().sort_values()[::-1].index:
    if i in many_model_manufacturers:
        all_mpg.append(i)

top5_mpg = all_mpg[:5]
print(top5_mpg)

['vw', 'honda', 'datsun', 'mazda', 'volkswagen']


In [ ]:
# 붓꽃 종별로 꽃잎길이 sepal_length, 꽃잎폭 sepal_width 꽃받침길이 petal_length, 꽃받침폭 petal_width 의 평균 , 표준편차 등 기초통계량을 구하시오
iris = sns.load_dataset('iris')
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [ ]:
iris.groupby('species').agg([np.mean, np.std]).round(3)

sepal_length        sepal_width  ... petal_length petal_width       
                   mean    std        mean  ...          std        mean    std
species                                     ...                                
setosa            5.006  0.352       3.428  ...        0.174       0.246  0.105
versicolor        5.936  0.516       2.770  ...        0.470       1.326  0.198
virginica         6.588  0.636       2.974  ...        0.552       2.026  0.275

[3 rows x 8 columns]

In [ ]:
# 3 분위수와 1 분위수의 차이보다 1.5 배가 크거나 작은 데이터는 이상치이다. 이 이상치를 제거하고 위의 4 가지 항목에 대해서 평균 , 표준편차를 구하시오

species_percentile = iris.groupby('species').apply(np.percentile, q=[0,25,50,75,100])
q1 = species_percentile.apply(lambda x : x[1])
q3 = species_percentile.apply(lambda x : x[3])

iris.loc[]




In [ ]:
q1

species
setosa        0.90
versicolor    1.95
virginica     2.50
dtype: float64

In [ ]:
iris.groupby('species').mean()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [ ]:
iris.groupby('species').apply(np.percentile, q=[0,25,50,75,100])

species
setosa        [0.1, 0.9, 2.0999999999999996, 4.3249999999999...
versicolor                           [1.0, 1.95, 3.3, 5.0, 7.0]
virginica                          [1.4, 2.5, 4.15, 6.025, 7.9]
dtype: object